In [230]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook

In [231]:
work_book = openpyxl.load_workbook('US_Airlines_DBs_2017_2016.xlsx')
sheet = work_book['dataset']

In [232]:
sheet.calculate_dimension()

'A1:I13'

In [233]:
from openpyxl.styles import numbers

for row in sheet['B2:G13']:
    for cell in row: 
        cell.value = int(cell.value)
        cell.number_format = numbers.FORMAT_NUMBER

In [234]:
from openpyxl.styles import PatternFill, colors
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule
from openpyxl.styles import Font, Color, colors

In [235]:
red_background = PatternFill(bgColor = colors.COLOR_INDEX[2])
diff_style = DifferentialStyle(fill = red_background, font = Font(color = colors.WHITE))
passenger_rule = Rule(type="expression", dxf = diff_style)

In [236]:
passenger_rule.formula = ["$F1<10000000"]

In [237]:
sheet.conditional_formatting.add('F2:F13', passenger_rule)

In [238]:
from openpyxl.formatting.rule import ColorScaleRule

In [239]:
for row in sheet['H2:I13']:
    for cell in row: 
        cell.value = float(cell.value)
        cell.number_format = numbers.FORMAT_NUMBER_COMMA_SEPARATED1

In [240]:
color_scale_rule = ColorScaleRule(start_type = "min",
                                 start_color = colors.COLOR_INDEX[5],
                                 end_type = "max",
                                 end_color = colors.COLOR_INDEX[6])

In [241]:
sheet.conditional_formatting.add("H2:I13", color_scale_rule)

In [242]:
work_book.save('US_Airlines_DBs_2017_2016_formatted.xlsx')

In [243]:
# Involutary Denied boarding Passengers bar chart

In [244]:
from openpyxl.chart import BarChart, Reference

In [245]:
bar_chart_1 = BarChart()
bar_chart_2 = BarChart()

In [246]:
bar_chart_1.type = "col"
bar_chart_1.title = "Involuntary Denied Boarding per US Airlines"
bar_chart_1.x_axis.title = "Airline"
bar_chart_1.y_axis.title = "Denied Boarding"

bar_chart_2.type = "col"
bar_chart_2.title = "Voluntary Denied Boarding per US Airlines"
bar_chart_2.x_axis.title = "Airline"
bar_chart_2.y_axis.title = "Denied Boarding"

In [247]:
data1 = Reference(worksheet = sheet, min_row = 1,
                max_row = sheet.max_row,
                min_col = 4, max_col = 5)

data2 = Reference(worksheet = sheet, min_row = 1,
                max_row = sheet.max_row,
                min_col = 2, max_col = 3)

In [248]:
cats = Reference(worksheet = sheet, min_row = 2,
                max_row = sheet.max_row, min_col = 1,
                max_col = 1)

In [249]:
bar_chart_1.add_data(data1, titles_from_data = True)
bar_chart_2.add_data(data2, titles_from_data = True)

In [250]:
bar_chart_1.set_categories(cats)
bar_chart_2.set_categories(cats)

In [251]:
bar_chart_1.width = 20
bar_chart_1.height = 10

bar_chart_2.width = 20
bar_chart_2.height = 10

In [252]:
sheet.add_chart(bar_chart_1, "A16")
sheet.add_chart(bar_chart_2, "G16")

In [253]:
work_book.save('US_Airlines_DBs_2017_2016_formatted.xlsx')

In [254]:
# Add data into the dataset  

In [255]:
path = 'US_Airlines_DBs_2017_2016_formatted.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

In [256]:
sheet['A']

(<Cell 'dataset'.A1>,
 <Cell 'dataset'.A2>,
 <Cell 'dataset'.A3>,
 <Cell 'dataset'.A4>,
 <Cell 'dataset'.A5>,
 <Cell 'dataset'.A6>,
 <Cell 'dataset'.A7>,
 <Cell 'dataset'.A8>,
 <Cell 'dataset'.A9>,
 <Cell 'dataset'.A10>,
 <Cell 'dataset'.A11>,
 <Cell 'dataset'.A12>,
 <Cell 'dataset'.A13>)

In [257]:
timeline_data = {}
for i in range(2, sheet.max_row-1):
    airline = sheet.cell(row = i, column = 1).value
    passengers_17 = str(sheet.cell(row = i, column = 6).value)
    passengers_16 = str(sheet.cell(row = i, column = 7).value)
    timeline_data[airline] = [ int(passengers_16),  int(passengers_17) ]

In [258]:
timeline_data

{'DELTA AIR LINES': [97237060, 99796155],
 'HAWAIIAN AIRLINES': [8154838, 8422734],
 'VIRGIN AMERICA': [5927938, 6090029],
 'UNITED AIRLINES': [64438132, 70303765],
 'ALASKA AIRLINES': [17725197, 18817924],
 'SKYWEST AIRLINES': [22575383, 24516354],
 'FRONTIER AIRLINES': [10895052, 12059943],
 'AMERICAN AIRLINES': [99348093, 98017132],
 'JETBLUE AIRWAYS': [25990828, 27255038],
 'SOUTHWEST AIRLINES': [112153048, 115988988]}

In [259]:
df = pd.DataFrame(timeline_data, index = ['2016','2017'])

In [260]:
df

,DELTA AIR LINES,HAWAIIAN AIRLINES,VIRGIN AMERICA,UNITED AIRLINES,ALASKA AIRLINES,SKYWEST AIRLINES,FRONTIER AIRLINES,AMERICAN AIRLINES,JETBLUE AIRWAYS,SOUTHWEST AIRLINES
2016,97237060,8154838,5927938,64438132,17725197,22575383,10895052,99348093,25990828,112153048
2017,99796155,8422734,6090029,70303765,18817924,24516354,12059943,98017132,27255038,115988988


In [261]:
df.to_excel(writer, sheet_name = 'enplanned_passengers')
writer.save()
writer.close()

In [262]:
# Enplanned Passengers line chart  

In [263]:
from openpyxl.chart import LineChart, Reference

In [264]:
book = openpyxl.load_workbook('US_Airlines_DBs_2017_2016_formatted.xlsx')
sheet = book['enplanned_passengers']

In [265]:
chart = LineChart()

In [266]:
values = Reference(sheet, min_col = 2, min_row = 1, max_col = sheet.max_column,
                  max_row = sheet.max_row)

In [267]:
chart.add_data(values, titles_from_data = True)

In [268]:
chart.title = "Enplanned Passengers per US Airlines"
chart.x_axis.title = "Year"
chart.y_axis.title = "Enplanned passengers"
chart.style = 10
chart.width = 25
chart.height = 12.5

In [269]:
dates = Reference(sheet, min_col = 1, min_row = 2,
                  max_col = 1, max_row = sheet.max_row)

In [270]:
chart.set_categories(dates)

In [271]:
styles = [
"FF7F50",
"FF4500",
"FFA500",
"FF1493",
"8B0000",
"FFD700",
"FF00FF",
"663399",
"32CD32",
"00FFFF"
]

In [272]:
len(chart.series)

10

In [273]:
i = 0
for s in chart.series:
    s.marker.symbol = "triangle"
    s.marker.graphicalProperties.solidFill = styles[i]
    s.marker.graphicalProperties.line.solidFill = styles[i]
        
    s.graphicalProperties.line.solidFill = styles[i]
    s.graphicalProperties.line.dashStyle = "solid"
    s.smooth = True
    
    i = i + 1 

In [274]:
sheet.add_chart(chart, "B5")

In [275]:
book.save('US_Airlines_DBs_2017_2016_formatted.xlsx')